In [2]:
import threading
import queue
import time
import random
from collections import defaultdict, deque

# Shared message queue for communication
message_queue = queue.Queue()

# Shared inventory and orders database
inventory_db = defaultdict(int)
orders_db = deque()

class Agent(threading.Thread):
    def __init__(self, name):
        super().__init__()
        self.name = name

    def send_message(self, message):
        message_queue.put((self.name, message))

    def receive_messages(self):
        messages = []
        while not message_queue.empty():
            sender, message = message_queue.get()
            if sender != self.name:
                messages.append((sender, message))
        return messages

    def run(self):
        raise NotImplementedError("This method should be overridden by subclasses")
        
class ReceivingAgent(Agent):
    def run(self):
        while True:
            # Simulate receiving inventory
            time.sleep(random.randint(1, 5))
            item = f"Item-{random.randint(100, 999)}"
            quantity = random.randint(1, 10)
            self.send_message(f"Received {item} {quantity}")
            print(f"{self.name}: Received {item} Quantity: {quantity}")

class StorageAgent(Agent):
    def __init__(self, name):
        super().__init__(name)
        self.storage = defaultdict(int)

    def run(self):
        while True:
            messages = self.receive_messages()
            for sender, message in messages:
                if "Received" in message:
                    _, item, quantity = message.split()
                    quantity = int(quantity)
                    self.storage[item] += quantity
                    inventory_db[item] += quantity
                    self.send_message(f"Stored {item} {quantity}")
                    print(f"{self.name}: Stored {item} Quantity: {quantity}, Total in Storage: {self.storage[item]}")

class PickingAgent(Agent):
    def run(self):
        while True:
            # Simulate picking an order
            time.sleep(random.randint(1, 5))
            if orders_db:
                order = orders_db.popleft()
                item, quantity = order
                if inventory_db[item] >= quantity:
                    inventory_db[item] -= quantity
                    self.send_message(f"Picked {item} {quantity}")
                    print(f"{self.name}: Picked {item} Quantity: {quantity}")
                else:
                    print(f"{self.name}: Insufficient stock for {item}")

class ShippingAgent(Agent):
    def run(self):
        while True:
            messages = self.receive_messages()
            for sender, message in messages:
                if "Picked" in message:
                    _, item, quantity = message.split()
                    self.send_message(f"Shipped {item} {quantity}")
                    print(f"{self.name}: Shipped {item} Quantity: {quantity}")

class OrderManager(threading.Thread):
    def run(self):
        while True:
            # Simulate creating an order
            time.sleep(random.randint(5, 10))
            item = f"Item-{random.randint(100, 999)}"
            quantity = random.randint(1, 5)
            orders_db.append((item, quantity))
            print(f"OrderManager: New order for {item} Quantity: {quantity}")


In [ ]:
if __name__ == "__main__":
    agents = [
        ReceivingAgent("ReceivingAgent"),
        StorageAgent("StorageAgent"),
        PickingAgent("PickingAgent"),
        ShippingAgent("ShippingAgent"),
        OrderManager()
    ]

    for agent in agents:
        agent.start()

    for agent in agents:
        agent.join()


ReceivingAgent: Received Item-807 Quantity: 1
ReceivingAgent: Received Item-934 Quantity: 4StorageAgent: Stored Item-934 Quantity: 4, Total in Storage: 4

ReceivingAgent: Received Item-964 Quantity: 9
OrderManager: New order for Item-398 Quantity: 3
ReceivingAgent: Received Item-781 Quantity: 6StorageAgent: Stored Item-781 Quantity: 6, Total in Storage: 6

PickingAgent: Insufficient stock for Item-398
OrderManager: New order for Item-315 Quantity: 1
ReceivingAgent: Received Item-275 Quantity: 10
PickingAgent: Insufficient stock for Item-315
ReceivingAgent: Received Item-317 Quantity: 3
StorageAgent: Stored Item-317 Quantity: 3, Total in Storage: 3
ReceivingAgent: Received Item-549 Quantity: 5StorageAgent: Stored Item-549 Quantity: 5, Total in Storage: 5

ReceivingAgent: Received Item-428 Quantity: 2StorageAgent: Stored Item-428 Quantity: 2, Total in Storage: 2

OrderManager: New order for Item-997 Quantity: 4
PickingAgent: Insufficient stock for Item-997
ReceivingAgent: Received Item-7